In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install neptune-client

In [2]:
%cd /content/drive/MyDrive/STUDY/Semester\ Four/COMP8420/A2/code

/content/drive/MyDrive/STUDY/Semester Four/COMP8420/A2/code


In [3]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import pandas as pd
import numpy as np
import read_data
import os
import shutil
from PIL import Image
import MyModels
import dataset
import random
import time

In [ ]:
# setup neptune
import neptune.new as neptune
api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiMjljZDQxYi1mODc1LTQ4ODktODYxMi0zNDcxZmIxY2E0YTgifQ=='

In [ ]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device = torch.device('cuda', 0)
else:
    device = torch.device('cpu')
print('use ', device)

# hyper-parameter
batch_size = 16
learning_rate = 1e-3
epoches = 300

seed = 666
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

use  cuda:0


In [ ]:
def train_model(k, model, datasets, dataLoaders, criterion, optimizer, scheduler, num_epoches, save_path):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epoches):
        if epoch%int(num_epoches/10) == 0:
            print('Epoch {}/{}'.format(epoch, num_epoches-1))
            print('-' * 10)

        # Each epoch has a training and test phase
        for phase in ['train', 'val']:
            since_epoch = time.time()
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)
            
            running_loss = 0.
            running_corrects = 0

            for data in dataLoaders[phase]:
                inputs, labels = data

                inputs = inputs.to(device)
                labels = labels.type(torch.LongTensor).to(device)

                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs.data, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc = running_corrects / len(datasets[phase])
            run['Fold {}/{}/{}'.format(k, phase, 'epoch_loss')].log(epoch_loss)
            run['Fold {}/{}/{}'.format(k, phase, 'epoch_acc')].log(epoch_acc)

            time_elapsed_epoch = time.time() - since_epoch
            # print('{} Loss: {:.4f} Acc: {:.4f} in {:.0f}m {:.0f}s'.format(
            #     phase, epoch_loss, epoch_acc, time_elapsed_epoch // 60, time_elapsed_epoch % 60))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        # print()
    # time_elapsed = time.time() - since
    # print('Training complete in {:.0f}m {:.0f}s'.format(
    #     time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    run['Fold{} best acc'.format(k)] = best_acc


    # load best model weights
    model.load_state_dict(best_model_wts)
    path = os.path.join(save_path, 'Fold{}_model.pth'.format(k))
    torch.save(model, path)
    return model


In [ ]:
def kfold_train():
    for k in range(5):
        print()
        print('Start fold {}'.format(k))
        print('-'*10)

        # models
        model = MyModels.Resnext50_32x4d(pretrained=True).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
        exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=int(epoches/3), gamma=0.1)

        dataset_train, dataset_test = dataset.get_crop_dataset(k)
        datasets = {
            'train': dataset_train,
            'val': dataset_test
        }

        trainloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=2)
        testloader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=2)
        dataLoaders = {
            'train': trainloader,
            'val': testloader
        }

        # create folder
        path = './models/pretrained_resnext50'
        if not os.path.exists(path):
            os.mkdir(path)
            print('made directorary ', path)

        model_ft = train_model(k, model, datasets, dataLoaders, criterion, optimizer, exp_lr_scheduler, num_epoches=epoches,
                               save_path = path)


In [ ]:
# setup neptune
run = neptune.init(project='panxnan/SFEW',
                api_token=api_token,
                )
kfold_train()

https://app.neptune.ai/panxnan/SFEW/e/SFEW-31

Start fold 0
----------


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth



made directorary  ./models/pretrained_resnext50
Epoch 0/299
----------


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 30/299
----------
Epoch 60/299
----------
Epoch 90/299
----------
Epoch 120/299
----------
Epoch 150/299
----------
Epoch 180/299
----------
Epoch 210/299
----------
Epoch 240/299
----------
Epoch 270/299
----------
Best val Acc: 0.518519

Start fold 1
----------
Epoch 0/299
----------
Epoch 30/299
----------
Epoch 60/299
----------
Epoch 90/299
----------
Epoch 120/299
----------
Epoch 150/299
----------
Epoch 180/299
----------
Epoch 210/299
----------
Epoch 240/299
----------
Epoch 270/299
----------
Best val Acc: 0.540741

Start fold 2
----------
Epoch 0/299
----------
Epoch 30/299
----------
Epoch 60/299
----------
Epoch 90/299
----------
Epoch 120/299
----------
Epoch 150/299
----------
Epoch 180/299
----------
Epoch 210/299
----------
Epoch 240/299
----------
Epoch 270/299
----------
Best val Acc: 0.525926

Start fold 3
----------
Epoch 0/299
----------
Epoch 30/299
----------
Epoch 60/299
----------
Epoch 90/299
----------
Epoch 120/299
----------
Epoch 150/299
----------

In [9]:
acc = {}
for k in range(2, 5):
    model = torch.load('models/cropfaces_pretrained_resnet50/Fold{}_model.pth'.format(k))
    model.train(False)
    _, dataset_test = dataset.get_crop_dataset(k)
    testloader = DataLoader(dataset_test, batch_size=16, shuffle=False, num_workers=2)

    class_correct = list(0 for i in range(7))
    class_total = list(0 for i in range(7))
    for data in testloader:
        inputs, labels = data
        labels = labels.cuda()
        inputs = inputs.cuda()
        outputs = model(inputs)
        _, pred = torch.max(outputs, 1)

        for label, prd in zip(labels, pred):
            if label == prd:
                class_correct[label] += 1
            class_total[label] +=1

    acc[k] = [class_correct, class_total]

In [8]:
# resnext_acc = acc
for correct, total in resnext_acc.values():
    ac = np.divide(np.asarray(correct), np.asarray(total))
    # print(ac)
    ov_ac = np.divide(np.sum(np.asarray(correct)), np.sum(np.asarray(total)))
    print(ov_ac)

0.34814814814814815
0.34074074074074073
0.37037037037037035
0.37777777777777777
0.3582089552238806


In [13]:
crop_acc = acc
for correct, total in crop_acc.values():
    ac = np.divide(np.asarray(correct), np.asarray(total))
    # print(ac)
    ov_ac = np.divide(np.sum(np.asarray(correct)), np.sum(np.asarray(total)))
    print(ov_ac)

0.4962962962962963
0.4222222222222222
0.5149253731343284
